https://www.kaggle.com/code/rkuo2000/chromadb-langchain-gpt4all

https://python.langchain.com/docs/integrations/llms/gpt4all

https://github.com/nomic-ai/gpt4all/tree/main/gpt4all-bindings/python

https://www.kaggle.com/code/gpreda/test-gpt4all-on-kaggle

In [1]:
import time
start = time.time()

In [2]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-7990b663-147a-1655-cb63-4f8fe8bf001f)
GPU 1: Tesla T4 (UUID: GPU-6d4174ef-2d96-1211-ea72-78ec71b64af7)


In [3]:
%%time

! pip install -qq -U langchain tiktoken pypdf chromadb faiss-gpu
! pip install -qq -U transformers InstructorEmbedding sentence_transformers==2.2.2
! pip install -qq -U bitsandbytes bitsandbytes-cuda117
! pip install -qq -U gpt4all

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 

In [4]:
#!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin models

In [5]:
import warnings
warnings.filterwarnings("ignore")

import os
import textwrap
import shutil
import pandas as pd
import glob

import langchain
from langchain.llms import HuggingFacePipeline

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

#quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

#os.environ['TRANSFORMERS_CACHE'] = 'cache/'

print(langchain.__version__)

2024-02-02 16:28:12.134822: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 16:28:12.134956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 16:28:12.300609: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


0.1.5


In [6]:
### Multi-document retriever
from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA, VectorDBQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
#from gpt4all import GPT4All
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [7]:
!wget https://gpt4all.io/models/gguf/starcoder-newbpe-q4_0.gguf

--2024-02-02 16:28:28--  https://gpt4all.io/models/gguf/starcoder-newbpe-q4_0.gguf
Resolving gpt4all.io (gpt4all.io)... 104.26.1.159, 104.26.0.159, 172.67.71.169, ...
Connecting to gpt4all.io (gpt4all.io)|104.26.1.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8987411904 (8.4G)
Saving to: 'starcoder-newbpe-q4_0.gguf'

starcoder-newbpe-q4 100%[===================>]   8.37G  43.3MB/s    in 2m 16s  

2024-02-02 16:30:44 (63.2 MB/s) - 'starcoder-newbpe-q4_0.gguf' saved [8987411904/8987411904]



In [8]:

llm = GPT4All(model="/kaggle/working/starcoder-newbpe-q4_0.gguf")

OSError: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /opt/conda/lib/python3.10/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllmodel.so)

In [ ]:
#print(llm.generate("The capital of France is ", max_tokens=1028))

In [ ]:
llm

In [ ]:
shutil.copytree('/kaggle/input/mkvc-12-finalmix/kaggle/working/mk-vectordb-chroma-twelve', '/kaggle/working/mk-vectordb-chroma')

In [ ]:
%%time

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="LazarusNLP/congen-simcse-indobert-base", model_kwargs={"device": "cuda"})

vectordb = Chroma(persist_directory='mk-vectordb-chroma',embedding_function=instructor_embeddings,)
#vectordb.persist()

In [ ]:
%%time
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
#retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 6, "lambda_mult": 0.25})

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                       chain_type="stuff", 
                                       retriever=retriever, 
                                       return_source_documents=True,
                                       verbose=False)

In [ ]:
qa_chain

In [ ]:
def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
def llm_ans(query):
    llm_response = qa_chain(query)
    ans = process_llm_response(llm_response)
    return ans

In [ ]:
#model

In [ ]:
dataset = pd.read_csv('/kaggle/input/mkri-qa-3/dataset.csv',sep=';')
dataset

In [ ]:
%%time
for x,y in zip(dataset.no,dataset.qa_comb):
    llm_time_start = time.time()
    print("Nomor: ",x)
    print(y)
    llm_ans(y)
    llm_time_end = time.time()
    print("Time: ",llm_time_end - llm_time_start)
    print("==============================================")

In [ ]:
end = time.time()
print(end - start)